---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [238]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    def handle_name(name):
        # remove parenthesis
        index = name.find('(')
        name = name if index==-1 else name[:index].strip()
        # remove brackets
        index = name.find('[')
        name = name if index==-1 else name[:index].strip()
        return name
    
    data_dict = {"State": [], "RegionName": []}
    with open("university_towns.txt", "r") as f:
        current_state = ""
        for line in f:
            line = line.strip()
            if line.find("[edit]") != -1:
                current_state = line
            else:
                data_dict["State"].append(handle_name(current_state))
                data_dict["RegionName"].append(handle_name(line))
                
    df = pd.DataFrame(data_dict, columns=["State", "RegionName"])
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [98]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_level = pd.read_excel("gdplev.xls")[['Unnamed: 4', 'Unnamed: 5']]
    GDP_level.rename(columns={'Unnamed: 4':'Quarter',
                           'Unnamed: 5':'GDP'}, inplace=True)
    GDP_level = GDP_level[7:][GDP_level["Quarter"] >= "2000q1"]
    # check for start year-quarter
    ans = None
    for i in range(0, len(GDP_level)-2):
        first_decline = GDP_level.iloc[i][1] > GDP_level.iloc[i+1][1]
        second_decline = GDP_level.iloc[i+1][1] > GDP_level.iloc[i+2][1]
        if first_decline and second_decline:
            ans = GDP_level.iloc[i][0]
            break
    return ans

get_recession_start()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


'2008q3'

In [104]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_level = pd.read_excel("gdplev.xls")[['Unnamed: 4', 'Unnamed: 5']]
    GDP_level.rename(columns={'Unnamed: 4':'Quarter',
                           'Unnamed: 5':'GDP'}, inplace=True)
    GDP_level = GDP_level[7:][GDP_level["Quarter"] >= get_recession_start()]
     # check for end year-quarter
    ans = None
    for i in range(0, len(GDP_level)-2):
        first_growth = GDP_level.iloc[i][1] < GDP_level.iloc[i+1][1]
        second_growth = GDP_level.iloc[i+1][1] < GDP_level.iloc[i+2][1]
        if first_growth and second_growth:
            ans = GDP_level.iloc[i+2][0]
            break
    return ans

get_recession_end()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


'2009q4'

In [119]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP_level = pd.read_excel("gdplev.xls")[['Unnamed: 4', 'Unnamed: 5']]
    GDP_level.rename(columns={'Unnamed: 4':'Quarter',
                           'Unnamed: 5':'GDP'}, inplace=True)
    GDP_level = GDP_level[7:][(GDP_level["Quarter"] >= get_recession_start()) &
                              (GDP_level["Quarter"] <= get_recession_end())]
    # check for bottom year-quarter
    bottom_value = GDP_level['GDP'].min()
    bottom = GDP_level[GDP_level['GDP'] == bottom_value].iloc[0][0]
    return bottom

get_recession_bottom()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


'2009q2'

In [181]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # prepare data
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    columns_to_keep = ['State', 'RegionName'] + housing_data.columns[51:].tolist()
    housing_data = housing_data[columns_to_keep]
    housing_data['State'] = housing_data['State'].map(states)
    housing_data.set_index(['State','RegionName'], inplace=True)
    # convert to quarters
    quarter = 1
    no_cols = len(housing_data.columns)
    for i in range(0, no_cols, 3):
        year_quarter_pair = housing_data.columns[i].split("-")[0] + "q" + str(quarter)
        cols_of_quarter = housing_data.columns[i:i+3].tolist()
        housing_data[year_quarter_pair] = housing_data[cols_of_quarter].mean(axis=1)
        quarter = 1 if quarter==4 else quarter+1
    
    return housing_data.drop(housing_data.columns[0:no_cols], axis=1) 

convert_housing_data_to_quarters().shape

(10730, 67)

In [239]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    before_recession_start = '2008q2'
    recession_bottom = get_recession_bottom()
    new_data = convert_housing_data_to_quarters()[[before_recession_start, recession_bottom]]
    new_data = new_data.reset_index()
    new_data.dropna(inplace=True)
    
    university_towns = get_list_of_university_towns()['RegionName'].tolist()    
    calc_price_ratio = lambda record: record[before_recession_start] / record[recession_bottom]
    new_data['Price ratio'] = new_data.apply(calc_price_ratio, axis=1)
    
    uni_towns = new_data[new_data['RegionName'].isin(university_towns)]['Price ratio']
    non_uni_towns = new_data[~new_data['RegionName'].isin(university_towns)]['Price ratio']
    #result = ttest_ind(uni_towns, non_uni_towns)
    p_value = ttest_ind(uni_towns, non_uni_towns)[1]
    result = (p_value < 0.01, p_value,
              "university town" if uni_towns.mean() < non_uni_towns.mean() else "non-university town")
    return result

run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(True, 6.7550789357461301e-05, 'university town')